# INSTALLING DEPENDENCIES

In [ ]:
# ! pip install transformers==4.0.0  Try this version of transformers if the latter version gives any errors
# !pip install torch==1.4.0  Try this version of transformers if the former version of PyTorch gives any errors

!pip install transformers==4.11.3
!pip install 'lightning-flash[text]' --upgrade
! pip install torch==1.5.0

# IMPORTING LIBRARIES AND DATASETS


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

# Check pytorch version
print(torch.__version__) 

In [ ]:
df = pd.read_fwf(r'/content/drive/MyDrive/AI/train_labelled.txt', header=None,encoding="ISO-8859-1",
                 error_bad_lines=False, usecols=[0,1])

df_test = pd.read_fwf(r'/content/drive/MyDrive/AI/val_labell.txt', header=None,encoding="ISO-8859-1",
                 error_bad_lines=False, usecols=[0,1])


# DATA PREPROCESSING

## RENAMING COLUMNS

In [ ]:
df.rename(columns={0: 'Sentiment',
                   1: 'Text'},
          inplace=True, errors='raise')

df_test.rename(columns={0: 'Sentiment',
                   1: 'Text'},
          inplace=True, errors='raise')

## DOING SOME REGEX FOR TEXT CLEANING

In [ ]:
import re
# remove all links
df['Text'] = df['Text'].str.replace(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', regex=True)

# remove all line breaks 
df['Text'] = df['Text'].str.replace(r'\n', '', regex=True)    
# text['tweet_text'] = text['tweet_text'].str.replace(r'\t', '', regex=True)
df['Text'] = df['Text'].str.replace(r'\r', '', regex=True)

# remove all emojis
df['Text'] = df['Text'].str.replace(r"["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", '', flags=re.UNICODE,regex=True)


# Remove double fullstops
df['Text'] = df['Text'].str.replace(r'\.+', '.', regex=True)


df['Text'] = df['Text'].str.replace(r"\.(?=\S)", '.', regex=True)

#Remove &amp sign
df['Text'] =df['Text'].str.replace(r"&amp;", 'and', regex=True)

df['Text'] = df['Text'].str.replace(r'/\S*@\S*\s?/g', '', regex=True)# remove all emails

# Remove all usernames from the tweets column for better interpretability, AFTER ADDING TO ANOTHER COLUMN
df['Text'] = df['Text'].str.replace(r'([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)', '', regex=True)  # remove all emails


# Instead of the upper two regexes, lets use this. Itll keep the user names but just remove the @ symbol
df['Text'] = df['Text'].str.replace('@', '', regex=True)
df['Text']=df['Text'].str.capitalize()


# remove all links
df_test['Text'] = df_test['Text'].str.replace(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', '', regex=True)

# remove all line breaks 
df_test['Text'] = df_test['Text'].str.replace(r'\n', '', regex=True)    
# text['tweet_text'] = text['tweet_text'].str.replace(r'\t', '', regex=True)
df_test['Text'] = df_test['Text'].str.replace(r'\r', '', regex=True)

# remove all emojis
df_test['Text'] = df_test['Text'].str.replace(r"["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", '', flags=re.UNICODE,regex=True)


# Remove double fullstops
df_test['Text'] = df_test['Text'].str.replace(r'\.+', '.', regex=True)

df_test['Text'] = df_test['Text'].str.replace(r"\.(?=\S)", '.', regex=True)
df_test['Text'] = df_test['Text'].str.replace(r'\â€”', 'and', regex=True)


df_test['Text'] = df_test['Text'].str.replace(r'/\S*@\S*\s?/g', '', regex=True)  # remove all emails

# Remove all usernames from the tweets column for better interpretability, AFTER ADDING TO ANOTHER COLUMN
df_test['Text'] = df_test['Text'].str.replace(r'([@#][A-Za-z0-9_]+)|(\w+:\/\/\S+)', '', regex=True)  # remove all emails


# Instead of the upper two regexes, lets use this. Itll keep the user names but just remove the @ symbol
df_test['Text'] = df_test['Text'].str.replace('@', '', regex=True)
df_test['Text']=df_test['Text'].str.capitalize()


In [ ]:
df.Sentiment.unique()

## RENAMING LABELS

In [ ]:
df['Sentiment'] = df['Sentiment'].replace(['__label__1','__label__2', '__label__3', '__label__4', '__label__5'],
                                              [0,1,2,3,4])

df_test['Sentiment'] = df_test['Sentiment'].replace(['__label__1','__label__2', '__label__3', '__label__4', '__label__5'],
                                              [0,1,2,3,4])


#  SPLITTING DATA INTO TRAINING AND TEST

In [ ]:
X_train = list(df['Text'])
X_test = list(df_test['Text'])
y_train = list(df['Sentiment'])
y_test = list(df_test['Sentiment'])

# CREATING THE TOKENIZER AND INSTANTIATING TRANSFORMER BERT MODEL

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=5)

In [ ]:
model = model.to('cuda')


# TEXT TOKENIZATION (NECESSARY TO TRANSFORM TEXT TO NUMBERS)

In [ ]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# CONVERTING TEXT AND LABELS TO TENSORS

In [ ]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])


In [ ]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_test)


# CREATING A COMPUTE FUNCTION THAT WILL CALCULATE THE MODEL ACCURACY, PRECISION, RECALL, AND F1-SCORE

In [ ]:
# You can check these functions too, if u want. Otherwise, simply delete this cell!
# def compute_metrics(p):
#     print(type(p))
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
# import numpy as np
# import evaluate
#  metric = evaluate.load("accuracy")

# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)


In [ ]:
CUDA_LAUNCH_BLOCKING=1
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## CREATING THE TRAINER USING THE TRAINER API

In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=15,
    per_device_train_batch_size=16,
    

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    
)

## MODEL TRAINING

In [ ]:
trainer.train()

# EVALUATE THE MODEL

In [ ]:
! pip install evaluate

In [ ]:
# This is causing some errors right now, ill fix it!
trainer.evaluate()

# SAVE THE MODEL

In [ ]:
trainer.save_model('custom_modelgggglatest')


# MAKE PREDICTIONS

## Write in the text


In [ ]:
text = "That was good point"

## Tokenize the text and send it to the model

In [ ]:
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
# print(outputs)

## Calculate the predictions for each labels, representing probabilities. Remember: 

__label__1 --> 0
<br></br>

__label__2 --> 1 
<br></br>

__label__3 --> 2,
<br></br>

__label__4 --> 3,
<br></br>

__label__5 --> 4
<br></br>

                                            

In [ ]:
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)


In [ ]:
predictions = predictions.cpu().detach().numpy()
predictions

## Extract the prediction with the highest probabilitly



In [ ]:
print("Probability is" , predictions[0][np.argmax(predictions)]*100 , " belonging to class " , np.argmax(predictions))